In [1]:
# prompt: load my google drive
import pandas as pd
import numpy as np
from datetime import datetime
pd.options.mode.copy_on_write = True

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
inpatient_readmits = pd.read_csv('/content/drive/MyDrive/Data/Output Data/inpatient_readmits.csv')
len(inpatient_readmits)

2797

In [3]:
member = pd.read_csv('/content/drive/MyDrive/Data/Output Data/MemberYear.csv')
len(member)

41379

In [4]:
member


,BENE_ID,STATE_CODE,COUNTY_CD,ZIP_CD,BENE_BIRTH_DT,SEX_IDENT_CD,BENE_RACE_CD,ESRD_IND,BENE_DEATH_DT,BENE_ENROLLMT_REF_YR,BENE_BIRTH_YR,AGE_ROUGH_CALC,AGE_GRP
0,-10000010254618,1,1500,36109,1999-08-16,1,1,0,NaN,2019,1999,20,ADULT BELOW 65
1,-10000010254647,1,1410,35756,1990-08-14,1,2,0,NaN,2019,1990,29,ADULT BELOW 65
2,-10000010254653,1,1400,36801,1982-03-18,2,1,0,NaN,2019,1982,37,ADULT BELOW 65
3,-10000010254656,1,1360,35216,1999-07-24,2,1,0,NaN,2019,1999,20,ADULT BELOW 65
4,-10000010254666,1,1440,35805,2009-02-23,2,4,0,NaN,2019,2009,10,CHILD
...,...,...,...,...,...,...,...,...,...,...,...,...,...
41374,-10000010288006,5,5330,95459,1942-09-14,1,1,0,NaN,2023,1942,81,ADULT 65+
41375,-10000010288007,5,5060,94598,1962-09-11,2,1,Y,NaN,2023,1962,61,ADULT BELOW 65
41376,-10000010288008,5,5430,92282,1945-12-26,1,5,0,NaN,2023,1945,78,ADULT 65+
41377,-10000010288009,5,5600,95380,1957-09-24,1,1,0,NaN,2023,1957,66,ADULT 65+


Join member and inpatient_results by BENE_ID and YR YR is called BENE_ENROLLMT_REF_YR.  But first I need to check whether a BENE_ID can appear in one year twice.  

In [5]:
duplicates = member[member.duplicated(subset=['BENE_ID', 'BENE_ENROLLMT_REF_YR'], keep=False)]
len(duplicates)


0

In [6]:
# prompt: Merge inpatient_readmits and member by BENE_ID and YR.  YR is called BENE_ENROLLMT_REF_YR in member.  Keep the rows in inpatient_readmits
inpatient_readmits2 = pd.merge(inpatient_readmits, member, how='left', left_on=['BENE_ID', 'YR'], right_on=['BENE_ID', 'BENE_ENROLLMT_REF_YR'])
len(inpatient_readmits2)


2797

In [7]:
inpatient_readmits2['BENE_ENROLLMT_REF_YR'].value_counts()


,count
BENE_ENROLLMT_REF_YR,
2022,907
2021,872
2020,842
2023,176


In [8]:
inpatient_readmits2['YR'].value_counts()

,count
YR,
2022,907
2021,872
2020,842
2023,176


In [9]:
inpatient_readmits2.columns

Index(['BENE_ID', 'CLM_ID', 'YR', 'LOS', 'DRG', 'PRNCPAL_DGNS_CD',
       'CLM_TOT_CHRG_AMT', 'ICD_Description', 'DRG_Description', 'ADMIT_DT',
       'DISCHARGE_DT', 'READMIT_FLAG', 'STATE_CODE', 'COUNTY_CD', 'ZIP_CD',
       'BENE_BIRTH_DT', 'SEX_IDENT_CD', 'BENE_RACE_CD', 'ESRD_IND',
       'BENE_DEATH_DT', 'BENE_ENROLLMT_REF_YR', 'BENE_BIRTH_YR',
       'AGE_ROUGH_CALC', 'AGE_GRP'],
      dtype='object')

Ready for logistic regression
Response is READMIT_FLAG
Independent Variables: LOS, PRNCPAL_DGNS_CD or DRG, COUNTY_CD, AGE_ROUGH_CALC, BENE_RACE_CD, ESRD_IND, SEX_IDENT_CD


DOING a big switcharoo with the Titanic dataset

In [10]:
import seaborn as sns
from seaborn import load_dataset
titanic = load_dataset('titanic')
titanic = titanic[['survived', 'pclass', 'sex', 'age']]
titanic['died'] = 1 - titanic['survived']
titanic = titanic.drop('survived', axis=1)



In [11]:
readmits_fake = titanic.rename(columns={'died': 'readmit', 'pclass': 'diagnosis'})

readmits_fake['diagnosis'] = readmits_fake['diagnosis'].astype('category')
readmits_fake['sex'] = readmits_fake['sex'].astype('category')
readmits_fake = readmits_fake.dropna()

In [12]:
# Center the 'age' column around 30
readmits_fake['age_centered'] = readmits_fake['age'] - 30
readmits_fake


,diagnosis,sex,age,readmit,age_centered
0,3,male,22.0,1,-8.0
1,1,female,38.0,0,8.0
2,3,female,26.0,0,-4.0
3,1,female,35.0,0,5.0
4,3,male,35.0,1,5.0
...,...,...,...,...,...
885,3,female,39.0,1,9.0
886,2,male,27.0,1,-3.0
887,1,female,19.0,0,-11.0
889,1,male,26.0,0,-4.0


In [13]:
import statsmodels.formula.api as sm
# Create dummy variables for categorical features
readmits_fake = pd.get_dummies(readmits_fake, columns=['diagnosis','sex'], drop_first=True)
formula = 'readmit ~ diagnosis_2 + diagnosis_3 + sex_male + age_centered'
model = sm.logit(formula=formula, data=readmits_fake).fit()
print(model.summary())


Optimization terminated successfully.
         Current function value: 0.453279
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                readmit   No. Observations:                  714
Model:                          Logit   Df Residuals:                      709
Method:                           MLE   Df Model:                            4
Date:                Fri, 04 Apr 2025   Pseudo R-squ.:                  0.3289
Time:                        18:04:59   Log-Likelihood:                -323.64
converged:                       True   LL-Null:                       -482.26
Covariance Type:            nonrobust   LLR p-value:                 2.074e-67
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept              -2.6675      0.261    -10.240      0.000      -3.178      -2.157
di

In [14]:
# prompt: extract coefficients only and then transform them into odds

# Extract coefficients from the model summary
coefficients = model.params
# Calculate odds ratios
odds_ratios = np.exp(coefficients)

print("Coefficients:")
print(coefficients)
print("\nOdds Ratios:")
odds_ratios


Coefficients:
Intercept             -2.667455
diagnosis_2[T.True]    1.309799
diagnosis_3[T.True]    2.580625
sex_male[T.True]       2.522781
age_centered           0.036985
dtype: float64

Odds Ratios:


,0
Intercept,0.069429
diagnosis_2[T.True],3.705430
diagnosis_3[T.True],13.205393
sex_male[T.True],12.463208
age_centered,1.037678


In [15]:
# prompt: attach predicted probabilities to the readmission dataset

# Predict probabilities using the fitted model
predicted_probabilities = model.predict(readmits_fake)

# Add the predicted probabilities as a new column to the DataFrame
readmits_fake['predicted_probability'] = predicted_probabilities



In [16]:
# prompt: how did you compute predicted probability?  show computations for first 5 rows of readmit_fake.

import numpy as np

# Extract coefficients from the model summary
coefficients = model.params

# Access the first five rows of the DataFrame
first_five_rows = readmits_fake.head()

# Manually calculate predicted probabilities for the first five rows
predicted_probs = []
for index, row in first_five_rows.iterrows():
  # Linear predictor
  linear_predictor = coefficients['Intercept'] + \
                    coefficients['diagnosis_2[T.True]'] * row['diagnosis_2'] + \
                    coefficients['diagnosis_3[T.True]'] * row['diagnosis_3'] + \
                    coefficients['sex_male[T.True]'] * row['sex_male'] + \
                    coefficients['age_centered'] * row['age_centered']

  # Predicted probability using the logistic function
  predicted_prob = 1 / (1 + np.exp(-linear_predictor))
  predicted_probs.append(predicted_prob)

# Print the calculated probabilities
print("Manually Calculated Predicted Probabilities for the first five rows:")
print(np.array(predicted_probs))

# Print the model predicted probabilities
print("\nModel Predicted Probabilities for the first five rows:")
print(first_five_rows['predicted_probability'].values)


Manually Calculated Predicted Probabilities for the first five rows:
[0.89473715 0.08536628 0.4415755  0.07709212 0.93219322]

Model Predicted Probabilities for the first five rows:
[0.89473715 0.08536628 0.4415755  0.07709212 0.93219322]


In [17]:
# prompt: write readmits_fake to csv file in '/content/drive/MyDrive/Data/Output Data'

# Save the DataFrame to a CSV file
readmits_fake.to_csv('/content/drive/MyDrive/Data/Output Data/readmits_fake.csv', index=False)


In [18]:
# prompt: write odds_ratio to a csv file

import pandas as pd
# Create a DataFrame from the odds ratios
odds_ratios_df = pd.DataFrame({'Variable': odds_ratios.index, 'Odds Ratio': odds_ratios.values})

# Save the DataFrame to a CSV file
odds_ratios_df.to_csv('/content/drive/MyDrive/Data/Output Data/odds_ratios.csv', index=False)


In [19]:
# prompt: write coefficents to a csv file

import pandas as pd
# Create a DataFrame from the coefficients
coefficients_df = pd.DataFrame({'Variable': coefficients.index, 'Coefficient': coefficients.values})

# Save the DataFrame to a CSV file
coefficients_df.to_csv('/content/drive/MyDrive/Data/Output Data/coefficients.csv', index=False)
